In [52]:
import os
import re

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

PATH_SAVE = "data"

<IPython.core.display.Javascript object>

In [53]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [190]:
def get_html(url):
    r = requests.get(url)
    return BeautifulSoup(r.text)

<IPython.core.display.Javascript object>

# Main page

In [84]:
full_page_html = get_html("https://www.letras.mus.br/mais-acessadas/bossa-nova/")
li_tags = full_page_html.find("ol", class_="top-list_art").find_all("li")

artist_list = []
for li in li_tags:
    artist_list.append(
        {
            "artist_name": li.text,
            "artist_page": f'https://www.letras.mus.br{li.find("a")["href"]}mais-tocadas.html',
        }
    )

<IPython.core.display.Javascript object>

In [189]:
artist_list

[{'artist_name': '  Vinicius de Moraes ',
  'artist_page': 'https://www.letras.mus.br/vinicius-de-moraes/mais-tocadas.html'},
 {'artist_name': '  Tom Jobim ',
  'artist_page': 'https://www.letras.mus.br/tom-jobim/mais-tocadas.html'},
 {'artist_name': '  João Gilberto ',
  'artist_page': 'https://www.letras.mus.br/joao-gilberto/mais-tocadas.html'},
 {'artist_name': '  Maysa ',
  'artist_page': 'https://www.letras.mus.br/maysa/mais-tocadas.html'},
 {'artist_name': '  Nara Leão ',
  'artist_page': 'https://www.letras.mus.br/nara-leao/mais-tocadas.html'},
 {'artist_name': '  Edu Lobo ',
  'artist_page': 'https://www.letras.mus.br/edu-lobo/mais-tocadas.html'},
 {'artist_name': '  Sérgio Mendes ',
  'artist_page': 'https://www.letras.mus.br/sergio-mendes/mais-tocadas.html'},
 {'artist_name': '  Baden Powell ',
  'artist_page': 'https://www.letras.mus.br/baden-powell/mais-tocadas.html'},
 {'artist_name': '  João Donato ',
  'artist_page': 'https://www.letras.mus.br/joao-donato/mais-tocadas.ht

<IPython.core.display.Javascript object>

# Song list page

In [187]:
def get_song_lines(full_page_html):
    return full_page_html.find("ul", class_="cnt-list-songs").find_all(
        "li", class_="cnt-list-row"
    )


def get_name_url(song_line):
    song_url = song_line["data-shareurl"]
    song_name = song_line.text
    return {"song_name": song_name, "song_url": song_url}

<IPython.core.display.Javascript object>

# Song Page

In [188]:
def extract_music_page(song_page):
    song_lyrics = song_page.find("div", class_="cnt-letra").get_text(" ")
    try:
        song_composition = (
            song_page.find("div", class_="letra-info_comp")
            .get_text("|")
            .split("|")[0]
            .split(":")[1]
        )
    except:
        song_composition = "Not known"
    return {"song_lyrics": song_lyrics, "song_composition": song_composition}

<IPython.core.display.Javascript object>

# getting everyting

In [173]:
%%time 
aux = []
for artist in artist_list:
    song_list = get_song_lines(get_html(artist["artist_page"]))
    song_refs = [get_name_url(song_line) for song_line in song_list]
    df_aux = pd.DataFrame(song_refs)
    df_aux["artist"] = artist["artist_name"]
    aux.append(df_aux)

aux = pd.concat(aux)
aux

,song_name,song_url,artist
0,Soneto de Fidelidade,https://www.letras.mus.br/vinicius-de-moraes/8...,Vinicius de Moraes
1,A Casa,https://www.letras.mus.br/vinicius-de-moraes/4...,Vinicius de Moraes
2,Onde Anda Você,https://www.letras.mus.br/vinicius-de-moraes/4...,Vinicius de Moraes
3,Poema Enjoadinho,https://www.letras.mus.br/vinicius-de-moraes/8...,Vinicius de Moraes
4,Samba da Bênção,https://www.letras.mus.br/vinicius-de-moraes/8...,Vinicius de Moraes
...,...,...,...
4,Set The Woods On Fire,https://www.letras.mus.br/art-in-manila/1173071/,Art In Manila
5,The Abomination,https://www.letras.mus.br/art-in-manila/1173074/,Art In Manila
6,The Game,https://www.letras.mus.br/art-in-manila/1173076/,Art In Manila
7,The Sweat Descends,https://www.letras.mus.br/art-in-manila/1173078/,Art In Manila


<IPython.core.display.Javascript object>

In [193]:
%%time
def merge_information(song_ref):
    try:
        song_ref_info = extract_music_page(get_html(song_ref["song_url"]))
    except:
        song_ref_info= {'song_lyrics':None, 'song_composition':None}
    return{**song_ref, **song_ref_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(line) for i, line in aux.iterrows())

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    1.7s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    3.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    4.6s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    7.5s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:   10.5s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   12.1s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   13.6s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   15.8s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   17.9s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   20.0s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   22.4s
[Parallel(

Wall time: 12min 28s


[Parallel(n_jobs=6)]: Done 6106 out of 6106 | elapsed: 12.5min finished


<IPython.core.display.Javascript object>

In [194]:
df = pd.DataFrame(aux_list)
df.drop("song_url", axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [195]:
from langdetect import detect, DetectorFactory


def detect_lang(string):
    try:
        return detect(string)
    except:
        return "Not known"


df["song_lang"] = df.song_lyrics.apply(detect_lang)

<IPython.core.display.Javascript object>

In [196]:
df.to_csv("bossa_nova_songs_dataset.csv", index=False)
df

,song_name,artist,song_lyrics,song_composition,song_lang
0,Soneto de Fidelidade,Vinicius de Moraes,"De tudo, ao meu amor serei atento antes E co...",Vinícius de Moraes.,pt
1,A Casa,Vinicius de Moraes,Era uma casa Muito engraçada Não tinha teto ...,Sergio Bardotti / Vinícius de Moraes.,pt
2,Onde Anda Você,Vinicius de Moraes,E por falar em saudade Onde anda você Onde a...,Hermano Silva / Vinícius de Moraes.,pt
3,Poema Enjoadinho,Vinicius de Moraes,Filhos... Filhos? Melhor não tê-los! Mas se ...,Vinícius de Moraes.,pt
4,Samba da Bênção,Vinicius de Moraes,É melhor ser alegre que ser triste Alegria é...,Baden Powell / Vinícius de Moraes.,pt
...,...,...,...,...,...
6101,Set The Woods On Fire,Art In Manila,We make things better We make things hard A ...,Art in Manila.,en
6102,The Abomination,Art In Manila,When i was a girl I witnessed an accident A...,Art in Manila.,en
6103,The Game,Art In Manila,These hits have made a bruise I wear to show...,Art in Manila.,en
6104,The Sweat Descends,Art In Manila,The sweat descends The sweat descends One co...,Not known,en


<IPython.core.display.Javascript object>